# A collection of examples showing how to carry out computations of 
# Protein-Ligand  Interaction Fingerprits, IFPs,  using MD-IFP tools but without using a trajectory object.

# IFPs can be generated for 

## (i) a PDB structure
## (ii) a single MD trajectory

    1. Computing interaction fingerprints (IFP) for
        -- a single structure prepared for MD simulations (HSP90; PDB ID 6EI5, dcd format)
        -- PDB structure
    2. Visualizing protein residues that are involved in protein-ligand interactions, including water-bridges
  
  

### Input data required:
    MD trajectory file of a complex
    pdb file of a complex (for example, generated from the first frame)
    pdb and mol2 file of a ligand separately 
    Data can be downloaded from https://zenodo.org/record/3755337#.XrF-iGgzaUk
    
    
### Packages required:
    numpy
    matplotlib
    MDAnalysis v. 20.1 or above
    pandas
    seaborn
    RDkit
    nglview
    code is written on Python 3.x and tested on the version 3.7



### v 1.0
    06.06.2020
    Copyright (c) 2020
    Released under the EUPL Licence, v1.2 or any higher version
    
### Authors: Daria Kokh & Fabian Ormersbach 
    Daria.Kokh@h-its.org
    Heidelberg Institute of Theoretical Studies (HITS, www.h-its.org)
    Schloss-Wolfsbrunnenweg 35
    69118 Heidelberg, Germany
    
    

##  Application example 

    Heat Shock Protein 90 (HSP90)

Import all of the required packages and scripts

In [25]:
import glob, os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nglview as nv
import MDAnalysis as mda
import warnings
from rdkit import Chem, RDConfig
from rdkit.Chem.rdMolChemicalFeatures import MolChemicalFeatureFactory
from Scripts.IFP_generation import *
from Scripts.Trajectories import *
#from Scripts.Complex_structure import *
#from Scripts.Clustering import *

%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Generation IFPs for a single HSP90 structure (PDB ID 6EI5) prepared for MD simulations 
### 1.1 find chemical properties of the ligand using RDKit 

Before we start with the IFP-Analysis we first need to extract the chemical information of the ligand.

In [26]:
# Define the path to the Ligand data, pdb file and mol2 file are needed
ligand_pdb = "Data/6EI5//INH.pdb"
ligand_mol2 = "Data/6EI5//moe.mol2"

# Get the atom names of the ligand
with open(ligand_pdb, 'r') as fasta_file: #Does ff stand for fasta file? 
    list_labels = [line.split()[2] for line in fasta_file.readlines() if line.split()[0]=='ATOM']

# Get the ligand features using RDKit feature factory
mol = Chem.rdmolfiles.MolFromMol2File(ligand_mol2, removeHs=False)
factory_name = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')
factory = Chem.ChemicalFeatures.BuildFeatureFactory(factory_name)
features = factory.GetFeaturesForMol(mol)

# Create dictionary that maps Atoms to their features
properties = {feature.GetFamily():[] for feature in features}
for feature in features:
    current_atoms = [list_labels[atom_id] for atom_id in list(feature.GetAtomIds())]
    for atom in current_atoms:
        properties[feature.GetFamily()].append(atom)
        
# Additionally we need to load our protein/ligand complex into a MDAnalysis universe
ref_pdb = 'Data/6EI5/ref-min.pdb'
u = mda.Universe(ref_pdb)

### 1.2 Generate PL IFPs 
Now that we have all the chemical information for our ligand, we can start the actual IFP analysis.  
The possible interactions with their corresponding abbreviations are shown in the table below.  
  
| Abbreviation |    Type of interaction   |
|:---------------|:---------------------------|
|   HY  | Hydrophobic |
|  IP  |       Salt bridge (positive charge) |
|  IN  |       Salt bridge (negative charge) |
|  AR  |   $\pi$-stacking  |
|    |  Cation-$\pi$  |
|    |  $\pi$-Cation  |
|    |  $\pi$-Amide  |
|  HL  |  Halogen-$\pi$  |
|    |  Halogen-carboxyl  |
|    |  Halogen-Amide  |
|    |  Halogen-Sulfur  |
|  HD  |  Hydrogen-bond donor  |
|  HA  |  Hydrogen-bond acceptor  |
|  WB  |  Water bridge  |

For this we use the IFP and table_combine functions contained in the IFP_generation script.\
The IFP function takes a variety of inputs.\
We need to supply our universe,  the ligand residue name and the properties of our ligand and set RE to false. The other settings we'll leave on the default for now.\
The IFP function returns three data frames: all IFPs properties except hydrogen bonds, hydrogen bonds only, and a waterbridges.
Then we can combine all three tables in one

In [27]:
df_prop,df_HB,df_WB = IFP(u,"INH",properties,RE=False)
df_prop_complete = table_combine(df_HB,df_WB,df_prop,"INH")

Start HB analysis 22:12:14.696077
Start WB analysis 22:12:15.411670
Start collecting IFPs:  22:12:17.571433
Start building IFP table:  22:12:20.880534
IFP database is ready  22:12:20.884529


This is the complete IFP database.\
It contains Waterbridges between the ligand and THR168, ASP77 and GLY81.\
Hydrogen bonds and aromatic contact with the ligand are constituted with ASP77, and GLY81/MET82 respectively.

In [28]:
df_prop_complete

,time,WB_LEU32,WB_SER36,HY_ALA39,HA_ASP77,WB_ASP77,HY_ILE80,WB_GLY81,HY_MET82,HY_LEU91,HY_ILE94,HY_PHE122,HD_THR168,WAT
0,0,1,1,1,1,1,1,1,1,1,1,1,1,4


### 1.4 Visualization of protein residues involved in the PL interactions
For the visualization in jp notebook we'll use NGLWidget.\
Let's first look at all residues participating in protein-ligand water bridges and H-bonds (taken from df_HB and df_WB).

In [29]:
view = nv.NGLWidget()

# We only want to see the protein and the ligand
u_reduced = u.select_atoms('protein or (resname INH WAT)')
w=nv.show_mdanalysis(u_reduced)
w.add_licorice("77 81 168")
w

NGLWidget()

Next, let us look at aromatic interactions. The following can be adapted to other interactions by changing "AR" to one of "HD", "HA", "IP", "IN", "WB", "HA".

In [30]:
view = nv.NGLWidget()
u_reduced = u.select_atoms('protein or (resname INH WAT)')
w = nv.show_mdanalysis(u_reduced)

residues = ''.join([' ' + str(ifp[6:]) for ifp in df_prop_complete.columns.tolist() 
                    if (ifp[:2] == 'AR')])
print('Showing residues', residues)
w.add_licorice(residues)
w

Showing residues 


NGLWidget()

## 2. Generation of PL IFPs for a single MD trajectory
### 2.1 Read a trajectory, compute PL IFPs and show results
This time we'll calculate the IFPs not for a static situation, but over a trajectory. The procedure stays mainly the same as the above, with a few tweaks.

In [31]:
# Set the file paths for all of the needed data
ref_pdb = "Data/6EI5/ref-min.pdb"
ligand_pdb = "Data/6EI5//INH.pdb"
ligand_mol2 = "Data/6EI5//moe.mol2"
traj = "Data/6EI5/amber2namd2.dcd"

# Initialize the MDAnalysis Universes (static and with trajectory) and calculate the radius of gyration 
ref = mda.Universe(ref_pdb)
Rgyr_t0 = ref.select_atoms("protein").radius_of_gyration()

u = mda.Universe(ref_pdb,traj) 
u_length = len(u.trajectory)

# Load part of the trajectory in the memory
u_reduced = u.select_atoms('protein or (resname INH WAT)')    
stepsize = 50
u_mem = mda.Merge(u_reduced).load_new(AnalysisFromFunction(
            lambda ag: ag.positions.copy(), u_reduced).run(
                start=0,stop=-1,step=stepsize).results,format=MemoryReader)

# Bring the trajectory into the right format
# One can save selected snapshots in the pdb format
save_pdb = False  

for i in range(0,len(u_mem.trajectory),1):
    u_mem.trajectory[i]  
    u_mem.dimensions = u.dimensions
    
    # Wrap all atoms back to the box, use radius of gyration for checking whether the procedure was successful
    Rgyr = pbc(u_mem,Rgyr_t0)

    # Calculate Root-mean-square deviation
    rmsd = superimpose_traj(ref,u_mem,["protein","resname INH"])
    
    print(i,"RMSD (protein , ligand):",rmsd,"traj length:",u_length)
    
    if save_pdb == True:
        all_write = u.select_atoms("all")
        all_write.write("/hits/fast/mcm/kokhda/TMP/"+"/namd_protein-"+str(i)+".pdb")

# Compute IFP:
# parameters used: 
#  WB_analysis=True  - find water-mediated protein-ligand contacts; 
#  RE = False - don't store unspecific prorein-ligand contacts
df_prop,df_HB,df_WB = IFP(u_mem, "INH", properties, WB_analysis=True, RE=False, Lipids=[])

# Build IFP table
. = table_combine(df_HB, df_WB, df_prop, "INH")

# Plot IFPs
Plot_IFP(df_prop_complete, contact_collection=None)

SyntaxError: invalid syntax (<ipython-input-31-a1523210d671>, line 48)

### Explanation for the plots:  
The first plot shows the devlopment of detected IFPs over the time of the trajetorie.  
The second plot shows all contacts between ligand and protein over the time of the trajectorie.  
The third plot shows the amount of molecules participating in the water shell of the ligand.

### 2.2 PL IFP table
Now we can take a look at the calculated tables.\
First we will look at water molecules participating in protein-ligand water bridges.
Not all frames are included in this table - only those where at least one water bridge found
"sele1*" is either ligand or water molecule that mediate ligand-protein interaction
"sele2*" is either protein or water

In [ ]:
df_WB

Then we can open the table with all hydrogen bonds

In [ ]:
df_HB

Finally, we can look at the combined table - it comprise all IFPs as well as protein resideus that participate in protein-ligand water bridges
This is a binary table with 1 

In [ ]:
df_prop_complete

### 2.3 Visualization of the IFP table 

Here is one more way to vizualize generated PL IFPs and water-mediated PL bonds

In [ ]:

Plot_IF_trajectory(df_prop_complete,ifp_type = np.asarray(['AR','HA','HD','IP','IN',"IO","WB"]),head_tail=10)

#### 2.4 Visualization of protein residues involved in PL interactions
For the last point of this part we are gonna visualize the residues that participate in the IFP.

In [ ]:
# obtain string that contains unique residues interacting with water
watr = ''.join([resid+' ' for resid in np.unique(water_bridges.sele2_resid.values.astype(str))])

# obtain residues that participate in aromatic interaction
residues = ''.join([str(ifp[6:]+' ') for ifp in df_prop_complete.columns.tolist() 
                    if (ifp[:2] == 'AR')])
print("Showing Residues: ", residues)

# Visualize the Complex

view = nv.NGLWidget()
protein = u_mem.select_atoms('protein or (resname INH WAT)')
w=nv.show_mdanalysis(protein)
w.add_licorice(watr+residues)
w

## 3. Generation of IFPs for a PDB file               (HSP90, PDB ID: 2YKI)

Lastlz, We will use the trajectory.py script in order to get the properties of our ligand and another IFP analysis.

### 3.1 IFP calculations

In [ ]:
# Define path to input data
ref_pdb = "Data/2YKI/2yki_MOE.pdb"
ligand_pdb_ = "Data/2YKI/ligand_2yki_MOE.pdb"
ligand_mol2 = "Data/2YKI/ligand_2yki_MOE.mol2"

# Load the pdb file into MDAnalysis universe
u = mda.Universe(ref_pdb)

# Define ligand residue name
sel_ligands = 'YKI'

# Get the chemical properties of the ligand with the ligand_properties function from the trajectories script
properties, mol = ligand_properties(ligand_pdb, ligand_mol2)
print(properties)

# IFP analysis, for reference look at 1. or 2.
df_prop,df_HB,df_WB = IFP(u,sel_ligands,properties,WB_analysis=True,RE=False)
df_prop_complete = table_combine(df_HB,df_WB,df_prop,sel_ligands)

# Summary for all interactions
ifp_type = ['HY','AR','WB','HA','HD']
at_type = [[],[],[],[],[]]
for v in df_prop_complete.columns.tolist():
    for i,t in enumerate(ifp_type):
        if v.find(t)>=0:
            at_type[i].append(v)
            
print('\n',10*'--','Contacts found:',10*'--')
print("\nHydrophobic contacts: ",at_type[0])
print("\nAromatic contacts: ",at_type[1])
print("\nWater Bridges: ",at_type[2])
print("\nHydrogen Acceptors: ",at_type[3])
print("\nHydrogen Donors: ",at_type[4])

In [ ]:
df_prop_complete

### 3.2 Visualization of PL interactions

In [ ]:
view = nv.NGLWidget()
protein = u.select_atoms('all')
w=nv.show_mdanalysis(protein)
residues = "".join([str(ifp[6:] + " ") for ifp in df_prop_complete.columns.tolist()])
print("Showing residues: ", residues)
w.add_licorice(residues)
w